# Setup

In [43]:
%cd  /home/tlm/Work/FYP-22-23/weee-preprocess

/home/tlm/Work/FYP-22-23/weee-preprocess


In [44]:
import os
import datetime
import numpy as np
import pandas as pd
import mne

In [45]:
DATASET_DIR = "./data/v1"
SAMPLING_RATE = 256 # Hz

# Prepare data

In [46]:
from utils.dataset import StudyInfoEncoder, DatasetVersion1
from utils.naming import standardize_column_names
from utils.resample import upsample
from utils.time import parse_simple_timedelta

# Process

In [47]:
dataset = DatasetVersion1()
activity_encoder = StudyInfoEncoder(os.path.join(DATASET_DIR, 'Study_Information.csv'))

In [48]:
def process_task(p):
    global activity_encoder
    df = dataset.muse_eeg(p)
    df.insert(1, 'user_id', p)
    activity_encoder.fit_activity_column(
        df, p, 
        timestamp_column='timestamp', 
        activity_column='session_type',
        activity_column_index=2,
    )
    df = standardize_column_names(df)
    print(f"Done processing participant {p:02d}")
    return df

In [49]:
# concat all participants row wise
from itertools import chain
muse_eeg = pd.concat([process_task(p) for p in chain(range(1, 2), range(3, 18))], axis=0, ignore_index=True)

Done processing participant 01
Done processing participant 03
Done processing participant 04
Done processing participant 05
Done processing participant 06
Done processing participant 07
Done processing participant 08
Done processing participant 09
Done processing participant 10
Done processing participant 11
Done processing participant 12
Done processing participant 13
Done processing participant 14
Done processing participant 15
Done processing participant 16
Done processing participant 17


# Windowing

In [50]:
WINDOW_SIZES = ['2s', '4s', '6s', '8s', '10s', '12s']
AGG_FUNCS = ['mean', 'std', 'min', 'max', 'median']
SKIP_FROM_START_SIT_FOR_EACH = '30s'

In [51]:
def create_non_overlapping_windows(window_size, agg_funcs, ignore_incomplete_windows=True):
    dfs = []
    for p, p_df in muse_eeg.groupby('user_id'):
        for a, pa_df in p_df.groupby('session_type'):
            # pre touch ups
            df = pa_df.sort_values('timestamp')
            if SKIP_FROM_START_SIT_FOR_EACH:
                lower_bound = df['timestamp'].iloc[0] + parse_simple_timedelta(SKIP_FROM_START_SIT_FOR_EACH)
                df = df[df['timestamp'] >= lower_bound]
            df = df.dropna(subset=['session_type']) # remove rows with nan activity
            if ignore_incomplete_windows:
                number_of_windows = len(df) // int(window_size[:-1])
                df = df.iloc[:number_of_windows*int(window_size[:-1])] # remove incomplete windows
            
            # create windows and aggregate
            agg_cols = {c: agg_funcs for c in df.columns if c not in ['timestamp', 'user_id', 'session_type']}
            agg_cols['timestamp'] =  ['first', 'last']
            windowed = df.groupby(pd.Grouper(key='timestamp', freq=window_size)).agg(agg_cols)

            # post touch ups
            windowed.columns = ['_'.join(col).strip() for col in windowed.columns.values]
            windowed = windowed.reset_index()
            windowed.insert(0, 'user_id', p)
            windowed.insert(1, 'session_type', a)
            windowed.insert(2, 'window_number', range(len(windowed)))
            windowed.insert(3, 'start_timestamp', windowed.pop('timestamp_first').dt.ceil(freq='s'))
            windowed.insert(4, 'end_timestamp', windowed.pop('timestamp_last').dt.ceil(freq='s'))
            windowed.drop(columns=['timestamp'], inplace=True)
            dfs.append(windowed)
    return pd.concat(dfs, axis=0, ignore_index=True)

In [52]:
%mkdir -p data/v2/MUSE
DATASET_DIR_V2 = "./data/v2"

In [53]:
for ws in WINDOW_SIZES:
    muse_eeg_windowed = create_non_overlapping_windows(ws, AGG_FUNCS)
    path = os.path.abspath(os.path.join(DATASET_DIR_V2, 'MUSE', f"muse_eeg_{ws}.csv"))
    muse_eeg_windowed.to_csv(path, index=False)
    print(f"Done processing window size {ws}")

Done processing window size 2s
Done processing window size 4s
Done processing window size 6s
Done processing window size 8s
Done processing window size 10s
Done processing window size 12s
